## Goal of the notebook

The scores for classical ML methods are quite low at around 65%. This is because these methods
are not accounting for the context in the text. So using a method which takes into account the
context would likely give better accuracy scores. So I will try to implement a basic BERT model in this notebook.

About BERT:

> BERT’s key technical innovation is applying the bidirectional training of Transformer, a popular attention model, to language modelling. This is in contrast to previous efforts which looked at a text sequence either from left to right or combined left-to-right and right-to-left training. 

We are basically using transfer learning for a specific task of sarcasm detection.

## How BERT works?

> *As opposed to directional models, which read the text input sequentially (left-to-right or right-to-left), the Transformer encoder reads the entire sequence of words at once. Therefore it is considered bidirectional, though it would be more accurate to say that it’s non-directional. This characteristic allows the model to learn the context of a word based on all of its surroundings (left and right of the word).*



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np 

from sklearn.model_selection import train_test_split  

import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras 
import tensorflow_text as text 

from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                            confusion_matrix, 
                            classification_report,ConfusionMatrixDisplay,
                            plot_confusion_matrix)

# from keras import backend as K
# from tensorflow.keras.preprocessing import sequence
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.optimizers import Adam
# from keras.layers import LSTM,Dense,Bidirectional,Input
# from keras.models import Model
# import torch
# import transformers 
# from transformers import AutoTokenizer


In [2]:
np.random.seed(73)#to maintain reproducibility

In [3]:
df =pd.read_json('../data/Sarcasm_Headlines_Dataset.json',lines=True)#using original dataset as BERT has preprocessing abilities
df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [4]:
X = df['headline']#creating X and y variables
y = df['is_sarcastic']


## Train Test Split

In [5]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y,)
X_train.head()

14075    tight wisconsin house primary too close to cal...
6861     labor dept. creates 20,000 new hobbies for nat...
20106    fired u.s. attorney preet bharara said to have...
8679     bosnian gum company introduces new war-flavore...
24542    adele sends her love to brussels with touching...
Name: headline, dtype: object

## BERT Model

In [6]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

2022-07-18 15:18:00.895287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

### Compilation

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # only using accuracy metric here
history = model.fit(X_train, y_train,validation_data=(X_test, y_test), epochs=5, batch_size = 32) # for 5 epochs, can be increased

Epoch 1/5
668/668 [==============================] - 11964s 18s/step - loss: 0.5328 - accuracy: 0.7457 - val_loss: 0.4920 - val_accuracy: 0.7757
Epoch 2/5
668/668 [==============================] - 4521s 7s/step - loss: 0.5048 - accuracy: 0.7608 - val_loss: 0.4908 - val_accuracy: 0.7864
Epoch 3/5
668/668 [==============================] - 8083s 12s/step - loss: 0.4908 - accuracy: 0.7697 - val_loss: 0.4712 - val_accuracy: 0.7694
Epoch 4/5
594/668 [=========================>....] - ETA: 6:35 - loss: 0.4799 - accuracy: 0.7759

In [ ]:
#prediction for test data
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
print(y_predicted[:5])

In [ ]:
#threshold
t = .5
# if a value is greater than or equal to 0.5 then it's a 1 else 0
y_predicted = [1 if x>=t else 0 for x in y_predicted]

### Model Summary

In [ ]:
model.summary()

### Confusion Matrix

In [ ]:
matrix = confusion_matrix(y_test, y_predicted)
print('Confusion Matrix: \n')
print(matrix)
# Convert to a pandas dataframe
confusion_df = pd.DataFrame(matrix, index=['0','1'], columns=['0','1'])

# Set the names of the x and y axis, this helps with the readability of the heatmap.
confusion_df.index.name = 'True Label'
confusion_df.columns.name = 'Predicted Label'
sns.heatmap(confusion_df, annot=True);

### Accuracy

In [ ]:
acc = accuracy_score(y_test, y_predicted)
print('accuracy:')
print(round(acc,2))

### Precision

In [ ]:
precision = precision_score(y_test, y_predicted)
print('Precision score:')
print(round(precision,2))

### Recall

In [ ]:
recall = recall_score(y_test, y_predicted)
print('Recall score:')
print(round(recall,2))

**Result:**

- **For 5 epochs accuracy is 77% which is 18% higher than baseline  and also significantly greater than the accuracy scores produced by classical ML methods.**
- **Context is taken into account which results in better scores. We can increase the number of epochs to possibly get even better scores.**

**Reference**:

- https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270    
- https://www.analyticsvidhya.com/blog/2021/12/text-classification-using-bert-and-tensorflow/ 
- https://towardsdatascience.com/multi-label-text-classification-using-bert-and-tensorflow-d2e88d8f488d